<a href="https://colab.research.google.com/github/rodannemann/imersaoAI2024/blob/main/ImersaoAI2024_RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Projeto ImersãoAI2024-RD
#### Classificação Inteligente de Posts em Mídias Sociais para Maximização de Receitas.
#### Um Estudo de Caso com um Grande Time de Futebol Brasileiro

## Inicializando o Modelo AI

In [39]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

from pathlib import Path
import hashlib
import google.generativeai as genai

import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# código para ocultar a chave de API
from google.colab import userdata
api_key_rod = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key_rod)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Leitura das Bases de Dados de Treino e Testes

In [71]:
LINK_BASE_TREINO = '/content/drive/MyDrive/Alura/base_treino_slim.csv'
LINK_BASE_TESTE = '/content/drive/MyDrive/Alura/base_teste_slim.csv'

# Carrega a base de treino
base_treino = pd.read_csv(LINK_BASE_TREINO)


# Carrega a base de teste
base_teste = pd.read_csv(LINK_BASE_TESTE)

base_treino.head()
#base_teste.head()

,Unnamed: 0,post_id,content_series,post_content
0,0,14299368,1,"Flamengo | Hoje, o Flamengo chora. Perdemos um..."
1,1,14329567,1,Flamengo | O número 7 é associado com a sabedo...
2,2,15784184,1,Flamengo | Fim de jogo. Bolívar 2 x 1 Flamengo...
3,3,15073739,1,Flamengo | O Clube de Regatas do Flamengo info...
4,4,16456062,1,Flamengo | FIMMMMM DE JOGO em Manaus! O MENGÃO...


## Treinando o Modelo

In [102]:
# Carrega 10x Posts do conteudo de FIM DE JOGO para treinar o modelo
base_treino_FimDeJogo = base_treino[base_treino["content_series"] == 1].head(10)
base_treino_FimDeJogo.head()

# Carrega 10x Posts do conteudo de ANIVERSARIO para treinar o modelo
base_treino_Aniversario = base_treino[base_treino["content_series"] == 0].head(10)

# Carrega 10x Posts do conteudo de ESCALACAO para treinar o modelo
base_treino_Escalacao = base_treino[base_treino["content_series"] == 2].head(10)

# Carrega 10x Posts do conteudo de COLETIVAS DE IMPRENSA para treinar o modelo
base_treino_ColetivaImprensa = base_treino[base_treino["content_series"] == 3].head(10)

# Faz o Merge de todos os posts de conteudo num unico dataframe
base_treino_prompt = [base_treino_FimDeJogo, base_treino_Aniversario, base_treino_Escalacao, base_treino_ColetivaImprensa]

# Adiciona os prompts para o modelo
# Loop through data and create entries
prompt_parts = []
image_parts = []
caption_parts = []
outputs = []
for post in base_treino_prompt:
  image_parts.append("/content/drive/MyDrive/Alura/treino/" + post["post_id"].astype(str) + ".jpg")
  caption_parts.append(post["post_content"])
  outputs.append(post["content_series"])

# Loop through data and create entries
for i in range(len(image_paths)):
  entry = [
      "input: ",
      f"genai.upload_file('{image_paths[i]}')",
      f"input 2: {captions[i]} | Instagram",
      f"output: {outputs[i]}"  # adjust output based on your data
  ]
  prompt_parts.append(entry)


In [103]:
# Carrega 10x Posts da base de teste que sejam de FIM DE JOGO
posts_fim_de_jogo = [ 30168922, 30244245, 30276517, 30276519, 30333143, 30333149, 30333150, 30378824, 30417583, 30800476]
base_teste_FimDeJogo = base_teste[base_teste["post_id"].isin(posts_fim_de_jogo)]

# Adiciona os prompts para o modelo
# Loop through data and create entries
prompt_parts = []
image_parts = []
caption_parts = []
for post in base_teste_FimDeJogo:
  image_file = "/content/drive/MyDrive/Alura/teste/" + post["post_id"].astype(str) + ".jpg"
  image_parts.append(image_file)
  caption_parts.append(post["post_content"])

# Loop through data and create entries
for i in range(len(image_paths)):
  entry = [
      "input: ",
      f"genai.upload_file('{image_paths[i]}')",
      f"input 2: {captions[i]} | Instagram",
      f"output: "  # output is empty on purpose
  ]
  prompt_parts.append(entry)

TypeError: string indices must be integers

In [78]:
# Passa Prompts e Requests para o Modelo
response = model.generate_content(prompt_parts)
print(response.text)

TypeError: Could not create `Blob`, expected `Blob`, `dict` or an `Image` type(`PIL.Image.Image` or `IPython.display.Image`).
Got a: <class 'list'>
Value: ['input: ', "genai.upload_file('0    /content/drive/MyDrive/Alura/14299368.jpg\n1    /content/drive/MyDrive/Alura/14329567.jpg\n2    /content/drive/MyDrive/Alura/15784184.jpg\n3    /content/drive/MyDrive/Alura/15073739.jpg\n4    /content/drive/MyDrive/Alura/16456062.jpg\n5    /content/drive/MyDrive/Alura/15737650.jpg\n6    /content/drive/MyDrive/Alura/16573425.jpg\n7    /content/drive/MyDrive/Alura/15214902.jpg\n8    /content/drive/MyDrive/Alura/16985251.jpg\n9    /content/drive/MyDrive/Alura/15900255.jpg\nName: post_id, dtype: object')", 'input 2: 0    Flamengo | Hoje, o Flamengo chora. Perdemos um...\n1    Flamengo | O número 7 é associado com a sabedo...\n2    Flamengo | Fim de jogo. Bolívar 2 x 1 Flamengo...\n3    Flamengo | O Clube de Regatas do Flamengo info...\n4    Flamengo | FIMMMMM DE JOGO em Manaus! O MENGÃO...\n5    Flame